In [2]:
import pandas as pd
import numpy as np


import torch
import torchvision as tv
import torchvision.transforms as transforms
import tqdm
from PIL import Image

import cv2
import os
import random

In [11]:
if not os.path.isdir('newimg'):
    os.makedirs('newimg')
WORKING_DIR = '/USER/Taeyun/'

In [111]:
df = pd.read_csv('train.csv')
train = df.loc[:580,:]

In [112]:
covid1 = train[train['COVID']==1]
covid0 = train[train['COVID']==0]

In [113]:
def make_vertical_new(img1,img2):
    upper = img1[:,:np.size(img1,1)//2,:]
    lower = img2[:,np.size(img2,1)//2:,:]
    return np.concatenate([upper,lower],axis=1)

In [156]:
def make_horizon_new(img1,img2):
    lefthalf = img1[:np.size(img1,0)//2,:,:]
    righthalf = img2[np.size(img2,1)//2:,:,:]
    return np.concatenate([lefthalf,righthalf],axis=0)

In [157]:
new_df = {"file_name":[],"COVID":[]}

In [9]:
os.path.join(WORKING_DIR,'train','a')

'/USER/Taeyun/train/a'

In [10]:
print(f'{WORKING_DIR}newmig')

/USER/Taeyun/newmig


In [158]:
for i in range(646,1147,4):
    #랜덤으로 이미지 뽑음
    Covid1_1,Covid1_2= random.choices(covid1['file_name'].to_list(),k=2)
    Covid0_1,Covid0_2 = random.choices(covid0['file_name'].to_list(),k=2)
    
    # 랜덤으로 뽑은 이미지 가져옴(np.array형태)
    Covid1_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid1_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid1_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

    Covid0_img1 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_1),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    Covid0_img2 = cv2.imread(os.path.join(WORKING_DIR,'train',Covid0_2),cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    
    #반복하면서 수직,수평으로 분할해서 라벨이 0,1인 이미지 2개생성
    # img를 폴더에 저장
    new_1_ver = make_vertical_new(Covid1_img1,Covid1_img2)
    new_0_ver = make_vertical_new(Covid0_img1,Covid0_img2)
    new_1_hor = make_horizon_new(Covid1_img1,Covid1_img2)
    new_0_hor = make_horizon_new(Covid0_img1,Covid0_img2)
    (Image.fromarray(new_1_ver)).save(f'{WORKING_DIR}newimg/{i}.png','png')
    (Image.fromarray(new_0_ver)).save(f'{WORKING_DIR}newimg/{i+1}.png','png')
    (Image.fromarray(new_1_hor)).save(f'{WORKING_DIR}newimg/{i+2}.png','png')
    (Image.fromarray(new_0_hor)).save(f'{WORKING_DIR}newimg/{i++3}.png','png')

    # 새로만들어진 img 라벨이 있는 csv작성
    new_df['file_name'].extend([f'{i}.png',f'{i+1}.png',f'{i+2}.png',f'{i+3}.png'])
    new_df['COVID'].extend([1,0,1,0])
        

In [159]:
new_df = pd.DataFrame(new_df)

In [160]:
new_df.to_csv(f'{WORKING_DIR}/new.csv',index=False)